# DeepFactor - Experiments

### Deep Factor (Wang et al.,  2019)

Hybrid method that uses *local* methods that are specific to each time series in the dataset as well as a *global* representation trained on the entire data. The latter is a RNN to learn universal pattern. The current version uses another RNN as local model -> **DeepFactor-RNN (DF-RNN)**

- [`DeepFactorEstimator`](http://gluon-ts.mxnet.io/master/api/gluonts/gluonts.model.deep_factor.html?highlight=deepfactorestimator#gluonts.model.deep_factor.DeepFactorEstimator)
- [Wang et al. (2019)](https://arxiv.org/abs/1905.12417)

In [3]:
# if version does not include DeepState, uncomment the following:
# !pip install git+https://github.com/awslabs/gluon-ts.git

In [13]:
import mxnet
mxnet.__version__

'1.4.1'

In [14]:
import gluonts
gluonts.__version__

'0.4.2'

In [15]:
import pprint
from functools import partial

# standard imports
import numpy as np
import pandas as pd

# vis imports 
%matplotlib inline
import matplotlib.pyplot as plt

# json
import json

# gluon data 
from gluonts.dataset.repository.datasets import get_dataset, dataset_recipes
from gluonts.dataset.util import to_pandas

# gluon imports
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator

# gluon distribution
from gluonts.distribution.piecewise_linear import PiecewiseLinearOutput
from gluonts.distribution.gaussian import GaussianOutput

# model imports 
# from gluonts.model.deepstate import DeepStateEstimator
from gluonts.model.deep_factor import DeepFactorEstimator
# from gluonts.model.deepar import DeepAREstimator

import mxnet as mx
from pprint import pprint

In [16]:
def deepfactor(seed=42, data="m4_daily", epochs=100, batches=50):
    
    mx.random.seed(seed)
    np.random.seed(seed)
    
    dataset = get_dataset(data)
    
    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=epochs,
        num_batches_per_epoch=batches,
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepFactorEstimator(
        trainer=trainer,
        cardinality=[cardinality],
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
#         num_layers_global=2,
#         num_layers_local=2
    )

    predictor = estimator.train(dataset.train)

    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )
    metrics = ["MASE", "sMAPE", "MSIS", "wQuantileLoss[0.5]", "wQuantileLoss[0.9]"]
    output = {key: round(value, 8) for key, value in agg_metrics.items() if key in metrics}
    df = pd.DataFrame([output])

    return df

In [ ]:
results = pd.DataFrame()

for i in range(42, 45):
    print("Seed:", i)
    df = deepfactor(data="m4_monthly", seed=i, epochs=500, batches=100)
    pprint(df)
    results = results.append(df)

results

INFO:root:downloading and processing m4_monthly


Seed: 42
saving time-series into /home/ec2-user/.mxnet/gluon-ts/datasets/m4_monthly/train/data.json
saving time-series into /home/ec2-user/.mxnet/gluon-ts/datasets/m4_monthly/test/data.json


INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 100/100 [00:06<00:00, 15.67it/s, avg_epoch_loss=2.69e+7]
INFO:root:Epoch[0] Elapsed time 6.389 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=26857936.577778
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 100/100 [00:05<00:00, 16.88it/s, avg_epoch_loss=1.28e+7]
INFO:root:Epoch[1] Elapsed time 5.938 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=12783945.911111
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 100/100 [00:05<00:00, 16.85it/s, avg_epoch_loss=2.6e+6]
INFO:root:Epoch[2] Elapsed time 5.948 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=2596963.906667
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 100/100 [00:05<00:00, 17.94it/s, avg_epoch_loss=1.08e+6]
INFO:root:Epoch[3] Elapsed time 5.581 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=1078675.778889
INFO:root:Epoch[4] Learning rate is 0.001
100%|██████████| 1

100%|██████████| 100/100 [00:05<00:00, 19.43it/s, avg_epoch_loss=7.55e+3]
INFO:root:Epoch[70] Elapsed time 5.153 seconds
INFO:root:Epoch[70] Evaluation metric 'epoch_loss'=7546.747352
INFO:root:Epoch[71] Learning rate is 0.000125
100%|██████████| 100/100 [00:05<00:00, 17.61it/s, avg_epoch_loss=1.19e+4]
INFO:root:Epoch[71] Elapsed time 5.687 seconds
INFO:root:Epoch[71] Evaluation metric 'epoch_loss'=11939.226832
INFO:root:Epoch[72] Learning rate is 0.000125
100%|██████████| 100/100 [00:05<00:00, 19.60it/s, avg_epoch_loss=9.46e+3]
INFO:root:Epoch[72] Elapsed time 5.109 seconds
INFO:root:Epoch[72] Evaluation metric 'epoch_loss'=9459.056984
INFO:root:Epoch[73] Learning rate is 0.000125
100%|██████████| 100/100 [00:06<00:00, 14.38it/s, avg_epoch_loss=1.09e+4]
INFO:root:Epoch[73] Elapsed time 6.979 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=10859.480757
INFO:root:Epoch[74] Learning rate is 0.000125
100%|██████████| 100/100 [00:06<00:00, 16.18it/s, avg_epoch_loss=1.1e+4]
INFO:

INFO:root:Epoch[140] Evaluation metric 'epoch_loss'=44041.719115
INFO:root:Epoch[141] Learning rate is 5e-05
100%|██████████| 100/100 [00:06<00:00, 16.11it/s, avg_epoch_loss=1.4e+4]
INFO:root:Epoch[141] Elapsed time 6.215 seconds
INFO:root:Epoch[141] Evaluation metric 'epoch_loss'=14034.795803
INFO:root:Epoch[142] Learning rate is 5e-05
100%|██████████| 100/100 [00:07<00:00, 13.46it/s, avg_epoch_loss=1.87e+4]
INFO:root:Epoch[142] Elapsed time 7.443 seconds
INFO:root:Epoch[142] Evaluation metric 'epoch_loss'=18695.084023
INFO:root:Epoch[143] Learning rate is 5e-05
100%|██████████| 100/100 [00:06<00:00, 14.87it/s, avg_epoch_loss=2.75e+4]
INFO:root:Epoch[143] Elapsed time 6.738 seconds
INFO:root:Epoch[143] Evaluation metric 'epoch_loss'=27473.072346
INFO:root:Epoch[144] Learning rate is 5e-05
100%|██████████| 100/100 [00:05<00:00, 17.22it/s, avg_epoch_loss=1.99e+4]
INFO:root:Epoch[144] Elapsed time 5.811 seconds
INFO:root:Epoch[144] Evaluation metric 'epoch_loss'=19912.031185
INFO:root:Ep

...

INFO:root:Epoch[249] Evaluation metric 'epoch_loss'=5062.852339
INFO:root:Epoch[250] Learning rate is 5e-05
100%|██████████| 100/100 [00:05<00:00, 17.22it/s, avg_epoch_loss=3.16e+3]
INFO:root:Epoch[250] Elapsed time 5.810 seconds
INFO:root:Epoch[250] Evaluation metric 'epoch_loss'=3157.666038
INFO:root:Epoch[251] Learning rate is 5e-05
100%|██████████| 100/100 [00:05<00:00, 17.32it/s, avg_epoch_loss=6.82e+3]
INFO:root:Epoch[251] Elapsed time 5.785 seconds
INFO:root:Epoch[251] Evaluation metric 'epoch_loss'=6817.864261
INFO:root:Epoch[252] Learning rate is 5e-05
100%|██████████| 100/100 [00:04<00:00, 20.11it/s, avg_epoch_loss=3.48e+3]
INFO:root:Epoch[252] Elapsed time 4.980 seconds
INFO:root:Epoch[252] Evaluation metric 'epoch_loss'=3477.013110
INFO:root:Epoch[253] Learning rate is 5e-05
100%|██████████| 100/100 [00:04<00:00, 20.32it/s, avg_epoch_loss=3.02e+3]
INFO:root:Epoch[253] Elapsed time 4.928 seconds
INFO:root:Epoch[253] Evaluation metric 'epoch_loss'=3023.400332
INFO:root:Epoch[

INFO:root:Epoch[321] Learning rate is 5e-05
100%|██████████| 100/100 [00:04<00:00, 24.24it/s, avg_epoch_loss=1.34e+3]
INFO:root:Epoch[321] Elapsed time 4.147 seconds
INFO:root:Epoch[321] Evaluation metric 'epoch_loss'=1342.055754
INFO:root:Epoch[322] Learning rate is 5e-05
100%|██████████| 100/100 [00:05<00:00, 19.87it/s, avg_epoch_loss=2.48e+3]
INFO:root:Epoch[322] Elapsed time 5.042 seconds
INFO:root:Epoch[322] Evaluation metric 'epoch_loss'=2480.422679
INFO:root:Epoch[323] Learning rate is 5e-05
100%|██████████| 100/100 [00:04<00:00, 20.01it/s, avg_epoch_loss=3.46e+3]
INFO:root:Epoch[323] Elapsed time 5.002 seconds
INFO:root:Epoch[323] Evaluation metric 'epoch_loss'=3458.987054
INFO:root:Epoch[324] Learning rate is 5e-05
100%|██████████| 100/100 [00:05<00:00, 19.71it/s, avg_epoch_loss=3.55e+3]
INFO:root:Epoch[324] Elapsed time 5.089 seconds
INFO:root:Epoch[324] Evaluation metric 'epoch_loss'=3546.951542
INFO:root:Epoch[325] Learning rate is 5e-05
100%|██████████| 100/100 [00:04<00:0

INFO:root:Epoch[356] Evaluation metric 'epoch_loss'=2398.347100
INFO:root:Epoch[357] Learning rate is 5e-05
100%|██████████| 100/100 [00:06<00:00, 15.91it/s, avg_epoch_loss=1.2e+3]
INFO:root:Epoch[357] Elapsed time 6.292 seconds
INFO:root:Epoch[357] Evaluation metric 'epoch_loss'=1197.205407
INFO:root:Epoch[358] Learning rate is 5e-05
100%|██████████| 100/100 [00:05<00:00, 16.98it/s, avg_epoch_loss=1.38e+3]
INFO:root:Epoch[358] Elapsed time 5.893 seconds
INFO:root:Epoch[358] Evaluation metric 'epoch_loss'=1377.208940
INFO:root:Epoch[359] Learning rate is 5e-05
100%|██████████| 100/100 [00:05<00:00, 19.48it/s, avg_epoch_loss=1.68e+3]
INFO:root:Epoch[359] Elapsed time 5.149 seconds
INFO:root:Epoch[359] Evaluation metric 'epoch_loss'=1675.321359
INFO:root:Epoch[360] Learning rate is 5e-05
100%|██████████| 100/100 [00:06<00:00, 16.45it/s, avg_epoch_loss=1.41e+3]
INFO:root:Epoch[360] Elapsed time 6.087 seconds
INFO:root:Epoch[360] Evaluation metric 'epoch_loss'=1413.451188
INFO:root:Epoch[3

100%|██████████| 100/100 [00:07<00:00, 13.99it/s, avg_epoch_loss=1.49e+3]
INFO:root:Epoch[428] Elapsed time 7.161 seconds
INFO:root:Epoch[428] Evaluation metric 'epoch_loss'=1493.630664
INFO:root:Epoch[429] Learning rate is 5e-05
100%|██████████| 100/100 [00:05<00:00, 17.93it/s, avg_epoch_loss=1.35e+3]
INFO:root:Epoch[429] Elapsed time 5.584 seconds
INFO:root:Epoch[429] Evaluation metric 'epoch_loss'=1347.369433
INFO:root:Epoch[430] Learning rate is 5e-05
100%|██████████| 100/100 [00:05<00:00, 18.40it/s, avg_epoch_loss=1.02e+3]
INFO:root:Epoch[430] Elapsed time 5.448 seconds
INFO:root:Epoch[430] Evaluation metric 'epoch_loss'=1020.429774
INFO:root:Epoch[431] Learning rate is 5e-05
100%|██████████| 100/100 [00:05<00:00, 18.51it/s, avg_epoch_loss=1.27e+3]
INFO:root:Epoch[431] Elapsed time 5.412 seconds
INFO:root:Epoch[431] Evaluation metric 'epoch_loss'=1270.502435
INFO:root:Epoch[432] Learning rate is 5e-05
100%|██████████| 100/100 [00:05<00:00, 17.31it/s, avg_epoch_loss=928]
INFO:root:

INFO:root:End model training
Running evaluation:  89%|████████▉ | 42878/48000 [19:57<02:18, 36.96it/s]